# Project :8 Working with Textual Data:  Text Classification

In the realm of entertainment, understanding audience sentiment towards movies is paramount for studios and production houses aiming to gauge the reception of their cinematic offerings. The IMDB movie review sentiment classification problem presents a crucial business challenge: accurately classifying movie reviews into positive or negative sentiments. What adds complexity to this task is the inherent variability in review lengths, the diverse vocabulary of words used, and the necessity for the model to discern the intricate long-term dependencies and contextual nuances embedded within the text.

  Create a first step document that lists the output of your exploratory analysis, any issues, or problems you may see with data that need follow-up, and some basic descriptive analysis that you think highlights important outcomes/findings from the data. Based on your findings, the next level of analysis will be charted out. Build a predictive model to classify the reviews into positive or negative. Perform a comparative study of several predictive models with various approaches and give your inferences accordingly.

**Dataset description:**  

There are 2 columns, review, and sentiment with 50000 records. ‘review’ column consists of imdb reviews. sentiment is the target variable with 2 classes (positive and negative)

**Dataset: IMDB Dataset.csv**

**Software Engineering aspect:**  

Utilize software engineering aspects while building the model using modular programming principles to organize your code into reusable functions or classes to enhance readability, maintainability, and collaboration.



**Initial Guidelines:**

1.	Ensure to follow to User Id’s provided by UNext for naming file as conventions.

2.	Create GitHub account and submit the GitHub link.

### General Instructions
- The cells in the Jupyter notebook can be executed any number of times for testing the solution
- Refrain from modifying the boilerplate code as it may lead to unexpected behavior 
- The solution is to be written between the comments `# code starts here` and `# code ends here`
- On completing all the questions, the assessment is to be submitted on moodle for evaluation
- Before submitting the assessment, there should be `no error` while executing the notebook. If there are any error causing code, please comment it.
- The kernel of the Jupyter notebook is to be set as `Python 3 (ipykernel)` if not set already
- Include imports as necessary
- For each of the task, `Note` section will provide you hints to solve the problem.
- Do not use `PRINT` statement inside the `Except` Block. Please use `return` statement only within the except block

In [6]:
#Required imports
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AdminUser\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AdminUser\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AdminUser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Task 1: Load the dataset and perform preliminary EDA with key observations and insights- (weightage - 20 marks)

#### T1.1: Load the IMDB Dataset using try and except blocks .           (weightage - 2 marks) (AE)

#### Note:
- Define a function named `load_the_dataset()` that attempts to load data from a CSV file named "IMDB Dataset.csv".
- Use a `try-except` block inside the function.
- Try to read the CSV file using `pd.read_csv()`.
- If successful, return the dataset (`df`).
- If there's an error (e.g., file not found), return the message "File not found. Please check the file path."

In [7]:
def load_the_dataset():
    try:
        df = pd.read_csv("IMDBDataset.csv")
        return df
    except :
        return "File not found. Please check the file path."

- After defining the function, call it to load the dataset and assign it to the variable `df`.
- Print the first few rows of the dataset using `print(df.head())`.

In [8]:
# store the result of the dataset
df=load_the_dataset()
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


#### T1.2: Check for distribution of target variable(percentage)(weightage - 2 marks)  (AE)             

#### Note:
- Define a function `target_class(df)` to analyze the distribution of a target variable ('sentiment') within a DataFrame (df).
- Inside the function, calculate the proportion of each unique value in the 'sentiment' column using the `value_counts` method with `normalize=True`.
- Multiply the proportions by 100 to obtain percentages.
- Return the calculated distribution.

In [9]:
def target_class(df):
    #code starts here
    target_dist = df['sentiment'].value_counts(normalize=True)*100
    #code ends
    return target_dist


- Call the function with `df` as argument to get `target_distribution`.
- Print `target_distribution`.

In [10]:
# store the result
target_distribution = target_class(df)
print(target_distribution)

sentiment
positive    50.0
negative    50.0
Name: proportion, dtype: float64


#### T1.3: Clean individual reviews: Remove all punctuations from words. Remove HTML tags. Remove words between square brackets. Remove all words that are not purely alphabetical characters. Convert all words to lowercase. Use error handling technique. (ME)

#### Note:
- Define a function `strip_html(text)` to remove HTML tags using BeautifulSoup.
- Define a function `remove_between_square_brackets(text)` to remove text within square brackets using regex.
- Define a function `denoise_text(text)` to apply `strip_html()` and `remove_between_square_brackets()` to lowercased text. --- - Handle exceptions using `try-except`.
- Define a function `remove_special_characters(text, remove_digits=True)` to remove special characters using regex.
- Define a function `remove_punctuation(text)` to remove punctuation using regex.
- Apply `denoise_text()`, `remove_special_characters()`, and `remove_punctuation()` functions to the 'review' column in DataFrame `df`.

In [11]:
def strip_html(text):
    soup = BeautifulSoup(str(text), "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
  try:
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = text.lower()
    return text
  except:
    return ("Failed to clean the text.")

#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
def remove_punctuation(text):
    # Define a regex pattern to match punctuation
    pattern = r'[^\w\s]'
    # Use the sub() function to replace punctuation with an empty string
    return re.sub(pattern, '', text)

# Assuming 'df' is your pandas DataFrame containing the IMDb dataset,
# and 'review_column' is the name of the column containing the reviews
df['review']=df['review'].apply(denoise_text)
df['review']=df['review'].apply(remove_special_characters)
df['review']=df['review'].apply(remove_punctuation)

C:\Users\AdminUser\AppData\Local\Temp\ipykernel_23484\2634588722.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(str(text), "html.parser")


- Print the first few rows of `df` to view the cleaned data.

In [12]:
df.head()

review sentiment
0  one of the other reviewers has mentioned that ...  positive
1  a wonderful little production the filming tech...  positive
2  i thought this was a wonderful way to spend ti...  positive
3  basically theres a family where a little boy j...  negative
4  petter matteis love in the time of money is a ...  positive

#### T1.4: Count the number of stopwords present. Remove all words that are known stop words.  (Use nltk)  (weightage - 2 marks)(AE)        

#### Note:
- Import necessary modules from NLTK for text preprocessing and download NLTK stopwords dataset if not already downloaded.
- Define two functions: `num_stopwords(review)` and `remove_stopwords(review)`.
- Tokenize the input review using NLTK's `word_tokenize` function.
- In `num_stopwords(review)`, count the number of stopwords present in the tokenized review.
- Return the count of stopwords.
- In `remove_stopwords(review)`, remove stopwords from the tokenized review.
- Join the cleaned tokens back into a string.
- Return the cleaned review.

In [13]:

# Download stopwords if not already downloaded
nltk.download('stopwords')

def num_stopwords(review):
    # Tokenize the review
    tokens = word_tokenize(review)
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    # Count the number of stopwords present
    num_stopwords = sum(1 for token in tokens if token.lower() in stop_words)
    return num_stopwords

def remove_stopwords(review):

    # Tokenize the review
    tokens = word_tokenize(review)
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    # Remove stopwords
    cleaned_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Join tokens back into a cleaned review
    cleaned_review = ' '.join(cleaned_tokens)

    return cleaned_review

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AdminUser\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


- Define a method `num_stopwords` to count the number of stopwords in each review of the 'review' column in DataFrame df.
- Apply the `num_stopwords` method to each review in the 'review' column using the `apply` function.
- Sum the total number of stopwords across all reviews.

In [14]:
# Apply function to the specified column in the DataFrame
df['num_stopwords'] = df['review'].apply(num_stopwords)
df['num_stopwords']

0        133
1         72
2         76
3         62
4         97
        ... 
49995    105
49996     54
49997    110
49998     97
49999     56
Name: num_stopwords, Length: 50000, dtype: int64

#### Range:
* num_stopwords: 530000 - 600000

- Define a function named `remove_stopwords` to remove common words (stopwords) from text data.
- Apply the `remove_stopwords` function to the 'review' column of DataFrame `df`.

In [15]:
df['review'] = df['review'].apply(remove_stopwords)

- Remove the 'num_stopwords' column from the DataFrame `df`.

In [16]:
# Drop the num_stopwords column
df.drop(columns=['num_stopwords'],inplace=True)

## T1.5: Remove all words that have a length of 1 character. Count the number of such words removed.Perform lemmatization to reduce words to their base form. (weightage – 4 marks)   (AE) & (ME)       

#### Note:
- Define a function `count_short_words(review)` to count short words in a review.
- Tokenize the review into words.
- Count words with a length of 1.
- Return the count of short words.

In [17]:
def count_short_words(review):#AE
    # Split the review into tokens
    tokens = word_tokenize(review)
    # Count words with length 1
    num_short_words = sum(1 for token in tokens if len(token) == 1)
    return num_short_words

- Define a function `count_short_words_apply` to count short words in a specified column of a DataFrame.
- Apply the `count_short_words` function to the specified column in the DataFrame and sum up the counts of short words.
- Return the total count of short words.

In [18]:
def count_short_words_apply(df, column_name):#AE
    # Apply count_short_words function to the specified column in the DataFrame
    total_short_words = df[column_name].apply(count_short_words).sum()
    return total_short_words

- Define a function called `remove_shortwords` to filter out short words from a given review.
- Tokenize the review into individual words using `word_tokenize`.
- Create a new list (`new`) containing only words with a length greater than 1, Join the filtered words back into a single string and return the filtered review string.

In [19]:
def remove_shortwords(review):#ME
  tokens = word_tokenize(review)
  new = [token for token in tokens if len(token)>1]
  return " ".join(new)

- Apply the function `count_short_words_apply` to the DataFrame `df` using the column 'review'.
- Print the total number of words with a length of 1 character.

In [20]:
total_short_words = count_short_words_apply(df, 'review')#AE
print("Total number of words with length = 1 character:", total_short_words)

Total number of words with length = 1 character: 21415


Range:
* total_short_words : (20000 - 30000)

- Apply the `remove_shortwords` function to the 'review' column in DataFrame `df`.

In [21]:
df['review'] = df.review.apply(remove_shortwords)

- Define a function `simple_lemmatizer` to perform lemmatization on text data.
- Tokenize the input text into individual words.
- Perform part-of-speech (POS) tagging on the words to determine their grammatical category (noun, verb, adjective, adverb).
- Lemmatize each word based on its POS tag, using WordNet lemmatization.
- Join the lemmatized words back into a string.
- Apply the `simple_lemmatizer` function to each review in the 'review' column of DataFrame `df`.
- Print the first few rows of the DataFrame using df.head().

In [22]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  # Download POS tagger

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization on text
def simple_lemmatizer(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    # Perform POS tagging
    tagged_words = nltk.pos_tag(words)
    # Lemmatize each word based on its POS tag
    lemmatized_words = []
    for word, tag in tagged_words:
        if tag.startswith('NN'):  # Noun
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='n'))
        elif tag.startswith('VB'):  # Verb
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='v'))
        elif tag.startswith('JJ'):  # Adjective
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='a'))
        elif tag.startswith('RB'):  # Adverb
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='r'))
        else:
            lemmatized_words.append(word)  # Default to original word
    # Join the lemmatized words back into a string
    return ' '.join(lemmatized_words)

# Test the lemmatizer function with a sample text
df['review']= df['review'].apply(simple_lemmatizer)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AdminUser\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AdminUser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AdminUser\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
df.head()

review sentiment
0  one reviewer mention watch oz episode youll ho...  positive
1  wonderful little production film technique una...  positive
2  think wonderful way spend time hot summer week...  positive
3  basically there family little boy jake think t...  negative
4  petter matteis love time money visually stunni...  positive

#### T1.6: Create wordcloud for positive and negative sentiments.             (weightage - 4 marks)            (ME)


#### Note:
- Define a WordCloud object named `WC` with the width of 1000 pixels, height of 500 pixels, maximum words of 500, and minimum font size of 5.
- Generate a WordCloud for positive reviews by joining all the review text where the sentiment is positive.
- Display the generated WordCloud using matplotlib.

In [24]:
#word cloud for positive review words
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,10))
# positive_text=' '.join(df[df['sentiment'] == 'positive']['review'])
# WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
# positive_words=WC.generate(positive_text)
# plt.imshow(positive_words,interpolation='bilinear')
# plt.show()

- Define a method `create_wordcloud` to generate a word cloud for negative review words.
- Join all the negative review texts into a single string.
- Create a WordCloud object with specified parameters (width, height, max_words, min_font_size).
- Generate the word cloud using the negative review text.
- Display the generated word cloud using Matplotlib.

In [25]:
# #word cloud for negative review words
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,10))
# negative_text=' '.join(df[df['sentiment'] == 'negative']['review'])
# WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
# negative_words=WC.generate(negative_text)
# plt.imshow(negative_words,interpolation='bilinear')
# plt.show

# Task 2: Build a Neural Network Predictive Model with Randomized Search (weightage - 30 marks)      

#### T2.1: Load the cleaned dataset and divide it into predictor and target values (X & y) (weightage – 5 marks) (AE)

#### Note:
- Define a function `separate_data_and_target` to split a DataFrame into input features (X) and target variable (y).
- Inside the function, extract the 'review' column as input features (X) and the 'sentiment' column as the target variable (y).
- Return X and y as separate entities.

In [26]:
# Splitting into input features and output(target variable)
# Separate independent features and target variable
def separate_data_and_target(df):
    X = df['review']
    y = df['sentiment']
    return X,y

- Assign the features to variable X and the target variable to variable y.
- Print the first few rows of X and by using `X.head()` and `y.head()`.

In [27]:
X, y = separate_data_and_target(df)
print(X.shape)
print(y.shape)

(50000,)
(50000,)


#### T2.2: Handling categorical features: Use TF-IDF vectorizer to convert into numerical features. Convert target variable, sentiment positive to 1 and negative to 0. (weightage - 2 marks)    (AE and ME)

#### Note:
- Define a function named `label_encode` that converts 'positive' sentiment to 1 and other sentiments to 0.
- Use the `map` method to apply the `label_encode` function to the 'sentiment' column of DataFrame `df`.

In [28]:
def label_encode(sentiment):
    return 1 if sentiment == 'positive' else 0

df['sentiment'] = df['sentiment'].map(label_encode)

- Define a method `tfidf_vectorizer` to transform text data into TF-IDF vectors.
- Specify the maximum number of features to consider using the `max_features` parameter.
- Use the `fit_transform` method to transform the text data into TF-IDF vectors.
- Assign the transformed data to the variable `X_tfidf`.

In [29]:
#set max_features = 5000
from sklearn.feature_extraction.text import TfidfVectorizer
def vector(df):
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X= df['review']
    y= df['sentiment']
    X_tfidf = tfidf_vectorizer.fit_transform(X)
    return X_tfidf

X_tfidf = vector(df)   

- Define a method `convert_sparse_matrix_to_dataframe` to convert a sparse matrix to a DataFrame.
- Use `pd.DataFrame()` to create a DataFrame from the sparse matrix `X_tfidf`.
- Optionally, add the 'review' column to the DataFrame using `df['review'].values`.
- Print the DataFrame.

In [30]:
import pandas as pd

# Convert the sparse matrix to a DataFrame
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Optional: Add the 'review' column to the DataFrame if needed
X_tfidf_df['review'] = df['review'].values

# Print the DataFrame
print(X_tfidf_df)


NameError: name 'tfidf_vectorizer' is not defined

## T2.3: Split the dataset into train and test in the ratio of 80:20. (weightage – 5 marks) (ME)

#### Note:
- Write a method using the `train_test_split` function from the `sklearn.model_selection` module to split data into training and testing sets.
- Use the feature matrix `X_tfidf.toarray()` and target vector `y`. 
- Set the test size to 20% and use a random state of 42 for reproducibility. The method should return `X_train`, `X_test`, `y_train`, and `y_test` arrays.

In [ ]:
# split into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf.toarray(), y, test_size=0.2, random_state=42)

#### T2.4: Train a Sequence classifier using standard Machine learning algorithm(Logistic Regression)  to classify  documents as either positive or negative . (weightage - 5 marks) (ME)

**Model versioning:**

- Save the model as ‘first_model’ to a version control system GitHub using git commands for collaboration, tracking changes, and ensuring transparency in model development.

#### Note:
- Define a logistic regression model for classification (`lr`) using the sklearn library.
- Set regularization penalty as L2, maximum iterations as 500, regularization strength as 1, and random state as 42.
- Fit the logistic regression model (`lr_classifier`) using training data (`X_train` and `y_train`).

In [ ]:
#training the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,precision_score,recall_score,f1_score
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model
lr_classifier =lr.fit(X_train,y_train)


Logistic Regression model was used to train the model to predict the classes positive and negative. Overall accuracy is found to be 89%.

#### T2.5: Train Multilayer Perceptron (MLP) models to classify documents as either positive or negative. (weightage - 10 marks) (ME)

**Model versioning**
-Save the model as ‘second_model’ to a version control system GitHub using git commands

#### Note:
- Import the Sequential and Dense modules from keras.models.
- Define a Sequential model using `Sequential()`.
- Add a dense layer with 16 units and ReLU activation using `model_mlp.add(Dense(...))`.
- Add another dense layer with 8 units and ReLU activation.
- Add a dense layer with 1 unit and sigmoid activation.
- Compile the model with 'rmsprop' optimizer and binary crossentropy loss using `model_mlp.compile()`.
- Train the model using `fit()`, specifying input data (`X_train`) and labels (`y_train`), batch size (10), and number of epochs (15).

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model_mlp = Sequential()
model_mlp.add(Dense(units=16, activation='relu', input_dim=X_train.shape[1]))
model_mlp.add(Dense(units=8, activation='relu'))
model_mlp.add(Dense(units=1, activation='sigmoid'))
model_mlp.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model_mlp.fit(X_train, y_train, batch_size=10, epochs=15)


Multilayer Perceptron (MLP) model is used to classify  documents as either positive or negative. It shows accuracy as 88%. It can be improved further by hyperparameter tuning.

#### T2.6: Train a CNN with Embedding layer to classify documents as either positive or negative.  (weightage-5 marks)(ME)

**Model versioning**
-  Save the model as ‘third_model’ to a version control system GitHub using git commands

#### Note:
- Create a CNN model for text classification.
- Import required modules from Keras and TensorFlow.
- Set the maximum number of words and sequence length.
- Use Tokenizer to tokenize and convert text to integers.
- Pad sequences to ensure they're all the same length.
- Split the data into training and testing sets.
    - Define the CNN model's architecture:
        - Include an embedding layer for word embeddings.
        - Add a Conv1D layer with 128 filters and a kernel size of 5.
        - Include a GlobalMaxPooling1D layer to reduce dimensionality.
        - Add two Dense layers with 64 and 1 units respectively.
- Compile the model using the Adam optimizer and binary crossentropy loss.
- Train the model for 10 epochs with a batch size of 32 and validate the results.

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Tokenize and pad sequences
max_words = 10000
maxlen = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(sequences, maxlen=maxlen)

# Split the Data
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Build the CNN Model
embedding_dim = 50
vocab_size = max_words

model_cnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=maxlen),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the Model
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
history_cnn = model_cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


# Task 3: Evaluate the performance of the model using the right evaluation metrics.                                                                         (weightage - 25 marks)

#### T3.1 Bring the models from a GitHub using git commands and evaluate the model (weightage - 2marks) (ME)

#### Added Model

In [ ]:
#training the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,precision_score,recall_score,f1_score
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model
lr_classifier =lr.fit(X_train,y_train)

#### T3.2 Evaluate the Logistic Regression model with evaluation metrics accuracy and precision using sklearn library. (weightage-5 marks) (AE)

In [ ]:
#Predicting the model

def evaluate_classification(lr_classifier,y_true, X_test):
    accuracy , precision ,recall ,f1 = 0.0, 0.0 ,0.0 ,0.0
    # code starts here
    y_pred =lr_classifier.predict(X_test)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    # code ends here
    return accuracy,precision,recall,f1

- Call the function with appropriate test data.

In [ ]:
evaluate_classification(lr_classifier,y_test, X_test)

#### Ranges
* Accuracy : 0.45 - 1 (2M)
* Precision: 0.45-1 
* Recall: 0.55-1
* F1 Score: 0.5 - 1

#### T3.3 Using Lime/SHAP libraries, explain the prediction of your model and give inferences. (weightage-5 marks) (ME)

#### T3.4 For the trained MLP model used, specify the accuracy score, loss value, epochs and activation function used at the output layer of the model (weightage-8 marks)(AE)

#### Added model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
def function_model_mlp(X_train,y_train):
    model_mlp = Sequential()
    model_mlp.add(Dense(units=16, activation='relu', input_dim=X_train.shape[1]))
    model_mlp.add(Dense(units=8, activation='relu'))
    model_mlp.add(Dense(units=1, activation='sigmoid'))
    model_mlp.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    history = model_mlp.fit(X_train, y_train, batch_size=10, epochs=7,verbose=0)
    return model_mlp,history

#### Note:

Define a method named `evaluate_mlp` to assess the performance of a __Multi-layer Perceptron (MLP) model__.
Inside the method:

- Utilize the model's evaluate function to compute the loss and accuracy using the test data (X_test and y_test).
- Determine the number of epochs by calculating the length of the loss history.
- Extract the name of the output activation function used in the last layer of the model.
- Return the computed loss, accuracy, number of epochs, and the name of the output activation function.

Remember to:
- Input the trained MLP model, test data (X_test and y_test), and training history.
- Use the method as follows: loss, accuracy, epochs, output_activation_function = evaluate_mlp(model, X_test, y_test, history).

In [ ]:
#Evaluate the Model
def evaluate_mlp(model,X_test,y_test,history):
    loss, accuracy = model.evaluate(X_test, y_test)
    epochs = len(history.history['loss'])
    output_activation_function = model.layers[-1].activation.__name__
    return loss, accuracy,epochs,output_activation_function

Use method evaluate_mlp to assess the MLP model's performance, then print accuracy, loss, epochs, and output activation function.

In [ ]:
loss, accuracy,epochs,output_activation_function = evaluate_mlp(model_mlp,X_test,y_test,history)
print(f"Accuracy Score: {accuracy}")
print(f"Loss Value: {loss}")
print(f"Epochs: {epochs}")
print(f"Activation Function at Output Layer: {output_activation_function}")

#### Ranges
* Accuracy : 0.45 - 1 (2M)
* Loss Value : 0-0.7 (2M)
* Epochs: 1-70 (0 M)
* Activation Function at Output Layer : "sigmoid" (1M)

#### T3.5 For the Trained CNN with Embedding layer, Specify the accuracy score, loss value, epochs used .(weightage-5marks) (AE)

#### Added CNN Model

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

def function_model_cnn(X,y):


    # Tokenize and pad sequences
    max_words = 10000
    maxlen = 5000
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    X_padded = pad_sequences(sequences, maxlen=maxlen)

    # Split the Data
    X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

    # Build the CNN Model
    embedding_dim = 50
    vocab_size = max_words

    model_cnn = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Compile the Model
    model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the Model
    history_cnn = model_cnn.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1,verbose =0)

    return model_cnn,history_cnn

model_cnn,history_cnn = function_model_cnn(X,y)

#### Note:
Create a method named `evaluate_cnn` to assess the performance of a __Convolutional Neural Network (CNN) model.__
- Within the method:
    - Utilize the model's evaluate function to calculate the loss and accuracy using the test data (X_test and y_test).
    - Determine the number of epochs by extracting the length of the loss history from the training history.
    - Return the computed loss, accuracy, and number of epochs.

Remember to:
- Provide the trained CNN model, test data (X_test and y_test), and training history as input parameters.
- Utilize the method like this: loss, accuracy, epochs = evaluate_cnn(model, X_test, y_test, history).

In [ ]:
#Evaluate the Model
def evaluate_cnn(model_cnn,X_test,y_test,history):
    loss, accuracy = model_cnn.evaluate(X_test, y_test)
    epochs = len(history.history['loss'])
    return loss, accuracy,epochs

- Invoke the evaluate_mlp method with arguments (model_cnn, X_test, y_test, history_cnn).
- Print the accuracy score, loss value and the number of epochs.

In [ ]:
accuracy,loss,epochs = evaluate_cnn(model_cnn,X_test,y_test,history_cnn)
print(f"Accuracy Score: {accuracy}")
print(f"Loss Value: {loss}")
print(f"Epochs: {epochs}")

#### Ranges
* Accuracy : 0.8 - 1 (2.5 M)
* Loss Value : 0 -0.95 (2.5 M)
* Epochs: 1-70 (0 M)


#### T3.6 Implement the unit test case and deploy a model using Flask / Streamlit. (weightage-10 marks)(ME)

### Note:

- Import the necessary libraries: __keras.models__ for Sequential model and __keras.layers__ for Dense layers.
- Create a new Sequential model named __keras_model.__
- Add layers to the Sequential model and define their configurations (units, activation functions, input dimensions).
- Set the weights of the layers in the new model to be the same as the weights of an existing model (model).
- Save the Keras model to an HDF5 file named _'final_model.h5'.__

In [ ]:
from keras.models import Sequential, save_model
from keras.layers import Dense

# Create a new Sequential model
# keras_model = Sequential()

# Add layers to the Sequential model and set the weights
# keras_model.add(Dense(units=16, activation='relu', input_dim=X_train.shape[1]))
# keras_model.add(Dense(units=8, activation='relu'))
# keras_model.add(Dense(units=1, activation='sigmoid'))

# # Set the weights of the layers in the new model
# keras_model.set_weights(model_mlp.get_weights())

# # Save the Keras model to an HDF5 file
# keras_model.save('final_model.h5')


- Import the necessary module json for working with JSON data.
- Convert the tokenizer's configuration to JSON format using the __to_json()__ method.
- Open a file named __'imdb_tokenizer.json'__ in write mode and encode it in UTF-8.
- Write the JSON data into the file using __json.dumps()__ function, ensuring non-ASCII characters are handled properly.

In [ ]:
import json

# Save tokenizer configuration to JSON file
tokenizer_json = tokenizer.to_json()
with open('imdb_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))


**Final Submission guidelines:** 
1.	Download the Jupyter notebook in the format of html. 
2.	Upload it in the lumen (UNext LMS)
3.	Take a screenshot of T15 (Deployment) and upload it in the lumen. (UNext LMS)
4.	Summarized PPT/ PDF prepared in Task 4 to be uploaded in the lumen. (UNext LMS)